# LSTM (Long Short Term Memory)


In [1]:
import numpy as np
from keras.callbacks import (
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)
from keras.layers import LSTM, Dense, Input
from keras.losses import MeanSquaredError
from keras.metrics import MeanSquaredError as MSEMetric
from keras.models import Sequential
from keras.optimizers import Adam

In [2]:
# The Fibonacci Sequence
def fibonacci(n):
    """Generate Fibonacci sequence up to n terms."""
    fib_list = [0, 1]
    for i in range(2, n):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list[:n]


raw_seq = fibonacci(15)
n_steps = 5  # Use 5 previous numbers to predict the next
n_features = 1


# Function to convert a sequence into (X, y) samples
def create_sequences(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


X, y = create_sequences(raw_seq, n_steps)

# Reshape input to be [samples, timesteps, features] for LSTM
X = X.reshape((X.shape[0], X.shape[1], n_features))

print(f"Sample X (Input): {X[0].flatten()} -> y (Output): {y[0]}")
print(f"Sample X (Input): {X[-1].flatten()} -> y (Output): {y[-1]}")

Sample X (Input): [0 1 1 2 3] -> y (Output): 5
Sample X (Input): [ 34  55  89 144 233] -> y (Output): 377


In [3]:
lstm = Sequential(
    layers=[
        Input(shape=(n_steps, n_features)),
        LSTM(64),
        Dense(64),
        Dense(32),
        Dense(1),
    ],
    name="lstm",
)

In [4]:
lstm.layers

[<LSTM name=lstm, built=True>,
 <Dense name=dense, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>]

In [5]:
lstm.summary()

Model: "lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,169 (90.50 KB)

 Trainable params: 23,169 (90.50 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
lstm.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MSEMetric()])  # type: ignore

In [ ]:
early_stopping_callback = EarlyStopping(
    monitor="loss", patience=10, restore_best_weights=True
)
model_checkpoint_callback = ModelCheckpoint(
    filepath="../../Models/lstm.keras",
    monitor="loss",
    save_best_only=True,
    mode="min",
)
tensorboard_callback = TensorBoard(
    log_dir="../../Logs/lstm_logs",
    histogram_freq=1,
    write_images=True,
    write_steps_per_second=True,
)
# tensorboard --logdir="Logs/lstm_logs"

csvlogger_callback = CSVLogger("../../Logs/lstm_logs.csv")

reduce_lr_callback = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=5)

lstm_history = lstm.fit(
    X,
    y,
    epochs=200,
    callbacks=[
        early_stopping_callback,
        model_checkpoint_callback,
        tensorboard_callback,
        csvlogger_callback,
        reduce_lr_callback,
    ],
)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 23028.6348 - mean_squared_error: 23028.6348 - learning_rate: 0.0010
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 22934.3770 - mean_squared_error: 22934.3770 - learning_rate: 0.0010
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - loss: 22842.6484 - mean_squared_error: 22842.6484 - learning_rate: 0.0010
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 22752.1523 - mean_squared_error: 22752.1523 - learning_rate: 0.0010
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 22662.0723 - mean_squared_error: 22662.0723 - learning_rate: 0.0010
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 22571.8086 - mean_squared_error: 22571.8086 - learning_rate: 0.0010
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 22480.9238 - mean_squared_error: 22480.9238 - learning_rate: 0.0010
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 22389.0410 - mean_squared_error: 22389.0410 - learning_

In [8]:
# New input data (the last n_steps numbers)
x_input = np.array([55, 89, 144, 233, 377])

# Reshape the input for the model: (1 sample, n_steps timesteps, 1 feature)
x_input = x_input.reshape((1, n_steps, n_features))

# Make the prediction
yhat = lstm.predict(x_input)

print(f"\n--- Prediction Result ---")
print(f"Input Sequence: {x_input.flatten()}")
# Round the prediction as Fibonacci numbers are integers
print(f"Predicted Next Number: {round(yhat[0][0])}")
print(f"Expected Next Number: ", x_input.flatten()[-1] + x_input.flatten()[-2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step

--- Prediction Result ---
Input Sequence: [ 55  89 144 233 377]
Predicted Next Number: 381
Expected Next Number:  610
